In [1]:
# import tcia2nifti as t2n
import glob
import os
from tqdm import tqdm

import pandas as pd
import pathlib as plb
import tempfile
import dicom2nifti
import SimpleITK as sitk
import shutil
import nibabel as nib

import numpy as np
import pandas as pd

In [2]:
data_paths = glob.glob('d:/AutoPET_data/PET+SUV_dir/*/*/*/*/*')
print(f'total number of origin candi = {len(data_paths)}')
valid_candidates = [] 
cnt = 0 
for path in tqdm(data_paths):
    lower_path = path.lower()
    path_slice = lower_path.split('\\')
    if 'uncorrected' in path_slice[-2]:
        # print(path)
        continue
        
    if 'no' in path_slice[-2]:
        # print(path)
        continue
        
    if 'nac' in path_slice[-2]:
        # print(path)
        continue
    
    # if 'suv' in path_slice[-2]:
    #     print(path)
    #     cnt += 1 
        
    # print(path)
    valid_candidates.append(path)
    
print(cnt)
len(valid_candidates), valid_candidates[0]

total number of origin candi = 8702


100%|██████████| 8702/8702 [00:00<00:00, 1450323.19it/s]

0


(6082,
 'd:/AutoPET_data/PET+SUV_dir\\ACRIN-FLT-Breast\\ACRIN-FLT-Breast_001\\01-01-1960-345678-PET-CT STUDY-69206\\PET-CT SERIES-00458\\PET.nii.gz')

In [3]:
val_intensity_list = [] 
val_min = []
val_max = [] 
val_shape = []

for path in tqdm(valid_candidates):
    img = nib.load(path)
    img_data = img.get_fdata() 
    
    min, max = np.min(img_data), np.max(img_data)
    
    if min != 0: 
        
        if min < -10:
            continue
            # print(min, max, path)
        
        if abs(min-max) < 5:
            continue
            # print(min, max, path)
            
    if max < 10:
        continue
    
    if max > 200:
        continue
    
    val_intensity_list.append(path)
    val_min.append(min)
    val_max.append(max)
    val_shape.append(str(img_data.shape))
    
print(len(val_intensity_list)) # 1739, 2398

100%|██████████| 6082/6082 [15:17<00:00,  6.63it/s]  

2398


In [4]:
val_min = []
val_max = [] 
valid_candidates = [] 
ori_dim = []
ori_shape = [] 
new_shape = []

new_spacing = (2.03642,  2.03642, 3.)
for path in tqdm(val_intensity_list):
    volume = sitk.ReadImage(path)
    img = sitk.GetArrayFromImage(volume)
    original_spacing = volume.GetSpacing()
    original_size    = volume.GetSize()
    new_size = [int(round(osz*ospc/nspc)) for osz,ospc,nspc in zip(original_size, original_spacing, new_spacing)]
    corresponding_size = sitk.Resample(volume, new_size, sitk.Transform(), sitk.sitkLinear,
                         volume.GetOrigin(), new_spacing, volume.GetDirection(), 0,
                         volume.GetPixelID())
    # print(f'{original_spacing}, {original_size}-->{corresponding_size.GetSize()}')
    
    H, W, D = corresponding_size.GetSize() 
    if H < 128:
        continue
    
    if W < 128:
        continue
    
    if D < 96:
        continue
    
    min, max = np.min(img), np.max(img)
    
    valid_candidates.append(path)
    val_min.append(min)
    val_max.append(max)
    ori_dim.append(str(original_spacing))
    ori_shape.append(str(original_size))
    new_shape.append(str(corresponding_size.GetSize()))

100%|██████████| 2398/2398 [06:03<00:00,  6.60it/s]


In [5]:
len(valid_candidates)

2254

In [6]:
import pandas as pd 

df = pd.DataFrame(np.stack([val_min, val_max, ori_dim, ori_shape, new_shape, valid_candidates]).T, columns=['min','max','ori_dim', 'ori_shape','new_shape','path'])
df.to_csv('D:/git/autoPET/val_PET(128x128x96)_list.csv')


In [7]:
df = pd.read_csv('D:\\git\\autoPET\\val_PET(128x128x96)_list.csv')

In [10]:
df.path[0]

'd:/AutoPET_data/PET+SUV_dir\\ACRIN-FLT-Breast\\ACRIN-FLT-Breast_001\\01-04-1960-345678-PET-CT STUDY-28908\\PET-CT SERIES-79785\\SUV.nii.gz'

In [8]:
######### for valid_CT img
target_root_dir = "D:\\AutoPET_data\\valid_SUV"

for path in tqdm(df['path']):
    chunk = path.split('/') 
    
    relative = chunk[2]
    relative = relative.split('\\')
    
    
    target = '\\'.join(relative[1:-2]) # 이게 어떤 파일인지 굳이 안 필요함 
    filename = relative[-1]
    copy = os.path.join(target_root_dir, target)
    if not os.path.isdir(copy):
        os.makedirs(copy)
        
    shutil.copy(path, os.path.join(copy,filename))


100%|██████████| 2254/2254 [00:58<00:00, 38.70it/s]
